## 0. Imports

In [1]:
from copy import deepcopy
import cv2 as cv
import imageio
import matplotlib.pyplot as plt
import numpy as np

from MyPython import MyCV
import MyData
import MyML

## 1. Data Fetching

In [2]:
# Import Data
keys = ["X", "Mean", "Std"]
Numbers = {k:v for k,v in zip(keys, MyData.getNumbers())}
Letters = {k:v for k,v in zip(keys, MyData.getLetters())}

## 2. Utility Functions

In [3]:
# Utils
def verify(img): assert (img.shape in [(784,), (1,784)]), f"should only get a single image, not {img.shape = }"
def reshape(img): return img.reshape(28,28)
def scale(img): return cv.resize(img, dsize=(280,280), interpolation=cv.INTER_NEAREST)

# Image Display and Insight
def show_img(img, T=False):
    img = scale(reshape(img))
    if T: img = img.T
    MyCV.show_img(img)
def show_imgs(imgs): 
    for img in imgs: show_img(img)
def img_summary(img):
    verify(img)
    return {'mean':np.average(img), 'std':np.std(img), 'min':np.min(img), 'max':np.max(img)}

# Video Writing
DEFAULT_fps = 4
DEFAULT_mbs = 14
def save_video(filename, imgs, fps=DEFAULT_fps, mbs=DEFAULT_mbs):
    imageio.mimwrite(filename, imgs, fps=fps, macro_block_size=mbs)

## 3. Train Pre-Transfer Models

### Number GANs

In [4]:
# 1. Number Models
NumberModels = dict()
nums = (0,1,2,3,4,5)
for num in nums:
    # Initialize Model
    model = MyML.GenerativeAdversarialModel(
        Data = Numbers["X"][num],
        DataMean = Numbers["Mean"][num],
        DataStd = Numbers["Std"][num],
        no_std = True,
        Bounds = [0,255],
        GeneratorLayers = [
            #MyML.FullyConnectedLayer(784, 784),
            #MyML.LinearLayer(),
            MyML.FullyConnectedLayer(784, 784),
            MyML.LinearLayer()
        ],
        DescriminatorLayers = [
            #MyML.FullyConnectedLayer(784, 1),
            #MyML.LinearLayer(),
            MyML.FullyConnectedLayer(784, 1),
            MyML.SigmoidLayer()
        ]
    )
    # Train Model
    imgs = model.train(100, 10 ** -20, 10)
    # Same Training Process in Video
    save_video(f"videos/transfer/Pre-Trained-{num}.mp4", (scale(reshape(img)) for img in imgs), fps=10)
    # Store Model in Dictionary
    NumberModels[num] = model

100%|██████████| 100/100 [00:00<00:00, 1041.68it/s]


### Letter GANs

![EMNIST Letter Mappings](misc/imgs/EMNIST_Balanced_Mappings.PNG "EMNIST Letter Mappings")

In [5]:
# 2. Letter Models
LetterModels = dict()
letter_nums = [
    24, # O -> ascii(79) -> 24
    18, # I -> ascii(73) -> 18
    35, # Z -> ascii(90) -> 35
    11, # B -> ascii(66) -> 11
    28  # S -> ascii(83) -> 28
]
for num in letter_nums:
    # Initialize Model
    model = MyML.GenerativeAdversarialModel(
        Data = Letters["X"][num],
        DataMean = Letters["Mean"][num],
        DataStd = Letters["Std"][num],
        no_std = True,
        Bounds = [0,255],
        GeneratorLayers = [
            #MyML.FullyConnectedLayer(784, 50),
            #MyML.LinearLayer(),
            MyML.FullyConnectedLayer(784, 784),
            MyML.LinearLayer()
        ],
        DescriminatorLayers = [
            #MyML.FullyConnectedLayer(784, 50),
            #MyML.LinearLayer(),
            MyML.FullyConnectedLayer(784, 1),
            MyML.SigmoidLayer()
        ]
    )
    # Train Model
    imgs = model.train(100, 10 ** -20, 10)
    # Same Training Process in Video
    save_video(f"videos/transfer/Pre-Trained-{num}.mp4", (scale(reshape(img).T) for img in imgs), fps=10)
    # Store Model in Dictionary
    NumberModels[num] = model

100%|██████████| 100/100 [00:00<00:00, 581.40it/s]


### Accessing Pre-Trained Models

In [6]:
def getNumberModel(num): return deepcopy(NumberModels[num])
def getLetterModel(num): return deepcopy(LetterModels[num])

## 4. Transfer Learning

In [7]:
PositivePairs = {n:l for n,l in zip(nums[:4]+nums[5:], letter_nums)}
print(PositivePairs)

{0: 24, 1: 18, 2: 35, 3: 11, 5: 28}


In [8]:
num = 2
model_start_number = getNumberModel(num)
#model_start_number.GeneratorLayers[0].hold = True
model_start_number.DescriminatorLayers[0].hold = True
model_start_number.DescriminatorLayers[0].hold = True
model_start_number.new_data(
    Letters["X"][PositivePairs[num]],
    Letters["Mean"][PositivePairs[num]],
    Letters["Std"][PositivePairs[num]]
)
imgs = model_start_number.train(1000, 10 ** -8, 1)
save_video(f"positive-{num}-{PositivePairs[num]}.mp4", (scale(reshape(img).T) for img in imgs), fps=10)

100%|██████████| 1000/1000 [00:00<00:00, 2840.95it/s]


In [9]:
NegativePairs = {
    4: 28,
    2: 11,
    0: 18,
    1: 24,
    3: 35
}
print(NegativePairs)

{4: 28, 2: 11, 0: 18, 1: 24, 3: 35}


In [10]:
num = 3
model_start_number = getNumberModel(num)
#model_start_number.GeneratorLayers[0].hold = True
model_start_number.DescriminatorLayers[0].hold = True
model_start_number.DescriminatorLayers[0].hold = True
model_start_number.new_data(
    Letters["X"][NegativePairs[num]],
    Letters["Mean"][NegativePairs[num]],
    Letters["Std"][NegativePairs[num]]
)
imgs = model_start_number.train(1000, 10 ** -8, 1)
save_video(f"negative-{num}-{NegativePairs[num]}.mp4", (scale(reshape(img).T) for img in imgs), fps=10)

100%|██████████| 1000/1000 [00:00<00:00, 2898.57it/s]
